In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

## Train

In [30]:
train = pd.read_csv('../Train/train_set_geo_level_sum_base.csv')

train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,geo_level_1_id_sum,geo_level_2_id_sum,geo_level_3_id_sum,base_percentage,damage_grade
0,6,487,12198,2,30,6,5,1,1,0,...,0,0,0,0,1,24381,270,37,1.200000,3
1,8,900,2812,2,10,8,7,0,1,0,...,0,0,0,0,1,19080,199,16,1.142857,2
2,21,363,8973,2,10,5,5,0,1,0,...,0,0,0,0,1,14889,1760,136,1.000000,3
3,22,418,10694,2,10,6,5,0,1,0,...,1,0,0,0,1,6252,205,31,1.200000,2
4,11,131,1488,3,30,8,9,1,0,0,...,0,0,0,0,1,8220,1038,122,0.888889,3


In [31]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]

In [32]:
# Mismo modelo que xgboost_v5
xgb_model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=11, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
xgb_model.fit(X,y)

/home/ben/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:56:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=11, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
result = xgb_model.apply(X)

result = result.astype('int32')

result

array([[1218, 7826, 3804, ..., 3417, 2637, 3231],
       [ 408, 8000, 7670, ...,  958, 2463, 2699],
       [ 692, 8685, 7579, ...,  637, 2572, 2611],
       ...,
       [ 408, 8370, 7904, ...,  955, 3179, 3195],
       [4447, 8919,  928, ..., 2934, 2699,  516],
       [ 387, 8593, 7497, ...,  117, 3115, 2587]], dtype=int32)

In [34]:
result_df = pd.DataFrame(result)

result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,1218,7826,3804,5444,9359,7970,5526,9350,6352,5366,...,3420,3807,3852,3194,3623,2900,2988,3417,2637,3231
1,408,8000,7670,404,8221,7550,414,8374,5871,1191,...,3427,1048,3852,2997,650,2901,2994,958,2463,2699
2,692,8685,7579,680,8673,7474,696,8789,5758,1115,...,3353,2969,3876,2959,2857,2899,2969,637,2572,2611
3,4949,7074,7100,5011,7310,6880,5100,7571,4886,4884,...,3670,3540,3876,3014,3314,2899,3172,3209,2600,2828
4,951,7159,6486,917,7403,6193,1532,7742,5265,1416,...,3257,2550,3672,3036,1246,2902,2801,798,2987,3195


In [35]:
cols = list(result_df.columns)

new_cols = ["leaf_index_"+str(n) for n in cols]

new_cols

['leaf_index_0',
 'leaf_index_1',
 'leaf_index_2',
 'leaf_index_3',
 'leaf_index_4',
 'leaf_index_5',
 'leaf_index_6',
 'leaf_index_7',
 'leaf_index_8',
 'leaf_index_9',
 'leaf_index_10',
 'leaf_index_11',
 'leaf_index_12',
 'leaf_index_13',
 'leaf_index_14',
 'leaf_index_15',
 'leaf_index_16',
 'leaf_index_17',
 'leaf_index_18',
 'leaf_index_19',
 'leaf_index_20',
 'leaf_index_21',
 'leaf_index_22',
 'leaf_index_23',
 'leaf_index_24',
 'leaf_index_25',
 'leaf_index_26',
 'leaf_index_27',
 'leaf_index_28',
 'leaf_index_29',
 'leaf_index_30',
 'leaf_index_31',
 'leaf_index_32']

In [36]:
result_df.columns = new_cols

result_df

,leaf_index_0,leaf_index_1,leaf_index_2,leaf_index_3,leaf_index_4,leaf_index_5,leaf_index_6,leaf_index_7,leaf_index_8,leaf_index_9,...,leaf_index_23,leaf_index_24,leaf_index_25,leaf_index_26,leaf_index_27,leaf_index_28,leaf_index_29,leaf_index_30,leaf_index_31,leaf_index_32
0,1218,7826,3804,5444,9359,7970,5526,9350,6352,5366,...,3420,3807,3852,3194,3623,2900,2988,3417,2637,3231
1,408,8000,7670,404,8221,7550,414,8374,5871,1191,...,3427,1048,3852,2997,650,2901,2994,958,2463,2699
2,692,8685,7579,680,8673,7474,696,8789,5758,1115,...,3353,2969,3876,2959,2857,2899,2969,637,2572,2611
3,4949,7074,7100,5011,7310,6880,5100,7571,4886,4884,...,3670,3540,3876,3014,3314,2899,3172,3209,2600,2828
4,951,7159,6486,917,7403,6193,1532,7742,5265,1416,...,3257,2550,3672,3036,1246,2902,2801,798,2987,3195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,4940,7038,7104,4906,7308,6898,5020,7575,4944,4792,...,3640,3536,3277,3037,3352,2544,3167,3220,2451,3057
260597,2831,8371,7897,3955,8816,7794,2845,8909,6049,1163,...,3104,2812,3876,2644,2687,2899,2647,3349,2453,2696
260598,408,8370,7904,404,8859,7807,414,8971,6067,1191,...,3527,1535,3619,3158,1009,2825,3073,955,3179,3195
260599,4447,8919,928,4446,9713,260,4517,9831,198,4387,...,883,3239,3873,849,3025,2786,344,2934,2699,516


In [37]:
new_train = train.join(result_df)

new_train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,leaf_index_23,leaf_index_24,leaf_index_25,leaf_index_26,leaf_index_27,leaf_index_28,leaf_index_29,leaf_index_30,leaf_index_31,leaf_index_32
0,6,487,12198,2,30,6,5,1,1,0,...,3420,3807,3852,3194,3623,2900,2988,3417,2637,3231
1,8,900,2812,2,10,8,7,0,1,0,...,3427,1048,3852,2997,650,2901,2994,958,2463,2699
2,21,363,8973,2,10,5,5,0,1,0,...,3353,2969,3876,2959,2857,2899,2969,637,2572,2611
3,22,418,10694,2,10,6,5,0,1,0,...,3670,3540,3876,3014,3314,2899,3172,3209,2600,2828
4,11,131,1488,3,30,8,9,1,0,0,...,3257,2550,3672,3036,1246,2902,2801,798,2987,3195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,0,1,0,...,3640,3536,3277,3037,3352,2544,3167,3220,2451,3057
260597,17,715,2060,2,0,6,5,0,1,0,...,3104,2812,3876,2644,2687,2899,2647,3349,2453,2696
260598,17,51,8163,3,55,6,7,0,1,0,...,3527,1535,3619,3158,1009,2825,3073,955,3179,3195
260599,26,39,1851,2,10,14,6,0,0,0,...,883,3239,3873,849,3025,2786,344,2934,2699,516


In [38]:
cols = list(new_train.columns)
cols.remove('damage_grade')
cols.append('damage_grade')

new_train = new_train[cols]

new_train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,leaf_index_24,leaf_index_25,leaf_index_26,leaf_index_27,leaf_index_28,leaf_index_29,leaf_index_30,leaf_index_31,leaf_index_32,damage_grade
0,6,487,12198,2,30,6,5,1,1,0,...,3807,3852,3194,3623,2900,2988,3417,2637,3231,3
1,8,900,2812,2,10,8,7,0,1,0,...,1048,3852,2997,650,2901,2994,958,2463,2699,2
2,21,363,8973,2,10,5,5,0,1,0,...,2969,3876,2959,2857,2899,2969,637,2572,2611,3
3,22,418,10694,2,10,6,5,0,1,0,...,3540,3876,3014,3314,2899,3172,3209,2600,2828,2
4,11,131,1488,3,30,8,9,1,0,0,...,2550,3672,3036,1246,2902,2801,798,2987,3195,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,0,1,0,...,3536,3277,3037,3352,2544,3167,3220,2451,3057,2
260597,17,715,2060,2,0,6,5,0,1,0,...,2812,3876,2644,2687,2899,2647,3349,2453,2696,3
260598,17,51,8163,3,55,6,7,0,1,0,...,1535,3619,3158,1009,2825,3073,955,3179,3195,3
260599,26,39,1851,2,10,14,6,0,0,0,...,3239,3873,849,3025,2786,344,2934,2699,516,2


In [39]:
new_train.to_csv('../Train/train_set_geo_sum_base_leaf_index_xgb_v5.csv', index=False)

## Test

In [29]:
test = pd.read_csv('../Test/test_set_geo_level_sum_base.csv')

test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,geo_level_1_id_sum,geo_level_2_id_sum,geo_level_3_id_sum,base_percentage
0,17,596,11307,3,20,7,6,0,1,0,...,0,0,0,0,0,1,7452,45,4,1.166667
1,6,141,11987,2,25,13,5,0,1,0,...,0,0,0,0,0,1,8104,65,3,2.600000
2,22,19,10044,2,5,4,5,0,1,0,...,0,0,0,0,0,1,2106,17,4,0.800000
3,26,39,633,1,0,19,3,0,0,0,...,0,0,0,0,0,2,7387,1329,190,6.333333
4,17,289,7970,3,15,8,7,0,1,0,...,0,0,0,0,0,1,7452,62,7,1.142857


In [40]:
result = xgb_model.apply(test)

result = result.astype('int32')

result

array([[ 187, 6938, 7092, ...,  186, 3113, 3219],
       [ 187, 6510, 4829, ...,  186, 2567, 3231],
       [2499, 6993, 6749, ..., 3126, 2567, 2793],
       ...,
       [4797, 7023, 6758, ..., 3202, 2544, 2951],
       [ 187, 4379, 6954, ..., 3091, 2451, 2648],
       [4218, 7081, 6055, ..., 2932, 2478, 1063]], dtype=int32)

In [41]:
result_df = pd.DataFrame(result)

result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,187,6938,7092,101,1878,6876,102,1065,4845,195,...,3265,353,3876,3157,341,2899,2838,186,3113,3219
1,187,6510,4829,101,6832,6698,102,7081,4753,195,...,3254,353,3851,3157,341,2901,2798,186,2567,3231
2,2499,6993,6749,3446,7255,6501,4867,7498,4886,4650,...,3125,3435,3556,2667,3309,1365,2659,3126,2567,2793
3,4165,1134,6058,4210,3294,5936,4322,7983,6723,4194,...,1276,3220,3412,1697,3025,2631,3391,2927,1318,2345
4,187,6953,7089,101,7235,6869,102,4989,4845,195,...,3265,353,3873,2959,341,2901,2838,186,3104,2611


In [42]:
cols = list(result_df.columns)

new_cols = ["leaf_index_"+str(n) for n in cols]

result_df.columns = new_cols

result_df

,leaf_index_0,leaf_index_1,leaf_index_2,leaf_index_3,leaf_index_4,leaf_index_5,leaf_index_6,leaf_index_7,leaf_index_8,leaf_index_9,...,leaf_index_23,leaf_index_24,leaf_index_25,leaf_index_26,leaf_index_27,leaf_index_28,leaf_index_29,leaf_index_30,leaf_index_31,leaf_index_32
0,187,6938,7092,101,1878,6876,102,1065,4845,195,...,3265,353,3876,3157,341,2899,2838,186,3113,3219
1,187,6510,4829,101,6832,6698,102,7081,4753,195,...,3254,353,3851,3157,341,2901,2798,186,2567,3231
2,2499,6993,6749,3446,7255,6501,4867,7498,4886,4650,...,3125,3435,3556,2667,3309,1365,2659,3126,2567,2793
3,4165,1134,6058,4210,3294,5936,4322,7983,6723,4194,...,1276,3220,3412,1697,3025,2631,3391,2927,1318,2345
4,187,6953,7089,101,7235,6869,102,4989,4845,195,...,3265,353,3873,2959,341,2901,2838,186,3104,2611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,4880,6541,6954,4906,6832,6763,5020,7081,4758,4758,...,3262,3501,3852,3036,3209,2901,2793,3170,3048,3195
86864,187,6875,7042,101,7144,6837,102,7407,1325,195,...,3259,353,3608,3178,341,2900,2798,2500,3102,3236
86865,4797,7023,6758,4770,7301,6504,4887,7575,4934,4653,...,3630,3451,3277,2886,3325,2532,3167,3202,2544,2951
86866,187,4379,6954,101,6889,6770,102,7130,4762,195,...,3041,353,3793,2634,341,2901,2647,3091,2451,2648


In [44]:
new_test = test.join(result_df)

In [45]:
new_test

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,leaf_index_23,leaf_index_24,leaf_index_25,leaf_index_26,leaf_index_27,leaf_index_28,leaf_index_29,leaf_index_30,leaf_index_31,leaf_index_32
0,17,596,11307,3,20,7,6,0,1,0,...,3265,353,3876,3157,341,2899,2838,186,3113,3219
1,6,141,11987,2,25,13,5,0,1,0,...,3254,353,3851,3157,341,2901,2798,186,2567,3231
2,22,19,10044,2,5,4,5,0,1,0,...,3125,3435,3556,2667,3309,1365,2659,3126,2567,2793
3,26,39,633,1,0,19,3,0,0,0,...,1276,3220,3412,1697,3025,2631,3391,2927,1318,2345
4,17,289,7970,3,15,8,7,0,1,0,...,3265,353,3873,2959,341,2901,2838,186,3104,2611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,4,605,3623,3,70,20,6,0,1,0,...,3262,3501,3852,3036,3209,2901,2793,3170,3048,3195
86864,10,1407,11907,3,25,6,7,1,1,1,...,3259,353,3608,3178,341,2900,2798,2500,3102,3236
86865,22,1136,7712,1,50,3,3,0,1,0,...,3630,3451,3277,2886,3325,2532,3167,3202,2544,2951
86866,6,1041,912,2,5,9,5,1,1,0,...,3041,353,3793,2634,341,2901,2647,3091,2451,2648


In [46]:
new_test.to_csv('../Test/test_set_geo_sum_base_leaf_index_xgb_v5.csv', index=False)